In [53]:
from cv2 import *
import joblib
import os
import numpy as np
import sklearn
import time
import pandas as pd 

from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score


In [54]:
os.getcwd()
os.chdir('/Users/yueyingteng/Documents/2016.9/Big Data /kaggle')

In [55]:
ls ./test_crop

1-500/         allpic/        test_crop_501/


In [56]:
model = joblib.load('GBDT.pkl')
voc = joblib.load('fullvoc.pkl')

In [67]:
# all croped test image in folder called "allpic"
test_pic = []
d = [d for d in os.listdir('./test_crop/allpic') if not d == '.DS_Store']
for d in d:
    test_pic.append(os.path.join('./test_crop/allpic', d))

In [68]:
len(test_pic), type(test_pic), test_pic[0]

(854, list, './test_crop/allpic/img_00007.jpg')

In [52]:
feature_det = xfeatures2d.SIFT_create()

matcher = BFMatcher(NORM_L2)
# extract descriptor of the new images baesed on the constructed vocabulary
# bow_extract  =cv2.BOWImgDescriptorExtractor(descr_ext,matcher)
bow_extract  = BOWImgDescriptorExtractor(feature_det,matcher)

def getImagedata(feature_det,bow_extract,path):
    im = imread(path)
    featureset = bow_extract.compute(im, feature_det.detect(im))
    return featureset
# returned featureset contains normzlized histogram

bow_extract.setVocabulary(voc)

In [69]:
test = []  

start = time.time()
for path in test_pic:
    featureset = getImagedata(feature_det,bow_extract,path)
    test.append(featureset)
    
end = time.time()
print (end - start)/60

In [73]:
# np.array(test).shape 
# # (854, 1, 500)

(854, 1, 500)

In [74]:
test = np.array(test).reshape(len(np.array(test)), -1)
test.shape
## (854, 500)

(854, 500)

In [88]:
folders = [f for f in os.listdir('./fish') if not f.startswith('.')]
col = []
for folder in folders:
    col.append(folder)
    
col

['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

In [94]:
# all croped test image in folder called "allpic"
test_name= []
d = [d for d in os.listdir('./test_crop/allpic') if not d == '.DS_Store']
for d in d:
    test_name.append(d)

# test_name

In [101]:
pred = model.predict(test)
prob = model.predict_proba(test)
# ALB BET DOL LAG OTHER SHARK YFT : 0-6
prob.shape
sub = pd.DataFrame(prob)
sub.columns = col
# df.insert(idx, col_name, value)
sub.insert(0, 'image', test_name)

sub.to_csv('submission.csv', encoding='utf8', index=False)

In [102]:
! open submission.csv